In [0]:
#Loading the Data File
# File location and type
file_location1 = "/FileStore/tables/application_record.csv"
file_type1 = "csv"
 
## File location and type
file_location2 = "/FileStore/tables/credit_record.csv"
file_type2 = "csv"
 
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
 
# The applied options are for CSV files. For other file types, these will be ignored.
data_file1 = spark.read.format(file_type1) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location1)
 
 
# The applied options are for CSV files. For other file types, these will be ignored.
data_file2 = spark.read.format(file_type2) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location2)
 
display(data_file1)
display(data_file2)

In [0]:
#Processing Datafile2 to  get Card Status of ID

#Here we have made couple of assumptions based on the information we collected from banking and crdit card vendors for getting card status
#People who tend to have dues for more than 2 months their credit score till be more drstically and low credit score can lead to rejection of new credit cards.
data_file=data_file2.toPandas()
data_file=data_file.replace(["C","X","0","1","2","3","4","5"],[0,0,0,0,1,1,1,1])
data_file=data_file.groupby("ID").sum("STATUS").reset_index()
data_credit_recored_processed=spark.createDataFrame(data_file)

In [0]:
#importing the libabraries
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
 
#Selecting Required Columns 
first_col="ID"
data_file1=data_file1.join(data_file,data_file1.ID==process_data_frame.ID,"left")
data_file1=data_file1.select([cols for cols in data_file1.columns if cols not in first_col])
data_file1=data_file1.na.fill(value=0)
def Card_Status_finder(t):
    if t>0:
        return 0
    return 1
func_udf = udf(Card_Status_finder, IntegerType())
data_file1=data_file1.withColumn("CARD_STATUS",func_udf(data_file1.STATUS))

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
 
#Selecting Required Columns 
def GetAge(t):
    return int(abs(t)/365)
func_udf = udf(GetAge, IntegerType())
data_file1=data_file1.withColumn("Age",func_udf(data_file1.DAYS_BIRTH))


In [0]:
#Selecting required coulmns
data_file1.select(["CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","CNT_CHILDREN","AMT_INCOME_TOTAL","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","Age","DAYS_EMPLOYED","FLAG_MOBIL","FLAG_WORK_PHONE","FLAG_PHONE","FLAG_EMAIL","OCCUPATION_TYPE","CNT_FAM_MEMBERS","MONTHS_BALANCE","CARD_STATUS"]).show()


+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-------------------+---+-------------+----------+---------------+----------+----------+---------------+---------------+--------------+-----------+
|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|  NAME_HOUSING_TYPE|Age|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|MONTHS_BALANCE|CARD_STATUS|
+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-------------------+---+-------------+----------+---------------+----------+----------+---------------+---------------+--------------+-----------+
|          F|           Y|              Y|           2|        135000.0|             Working|Secondary / secon...|             Married|  House / apartment

In [0]:
#Importing required modules
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler,StringIndexer ,OneHotEncoder
from pyspark.ml import Pipeline


In [0]:
#Selecting rrequired column
data=data_file1.select(["CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","CNT_CHILDREN","AMT_INCOME_TOTAL","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","Age","DAYS_EMPLOYED","FLAG_MOBIL","FLAG_WORK_PHONE","FLAG_PHONE","FLAG_EMAIL","OCCUPATION_TYPE","CNT_FAM_MEMBERS","MONTHS_BALANCE","CARD_STATUS"])


In [0]:
#Splitting data into train and test
train_data,test_data=data.randomSplit([0.7,0.3])

In [0]:
#String Indexer to represent the string features in the model.
CODE_GENDER_indexer = StringIndexer(inputCol='CODE_GENDER',outputCol='CODE_GENDER_index',handleInvalid='keep')
FLAG_OWN_CAR_indexer = StringIndexer(inputCol='FLAG_OWN_CAR',outputCol='FLAG_OWN_CAR_index',handleInvalid='keep')
FLAG_OWN_REALTY_indexer = StringIndexer(inputCol='FLAG_OWN_REALTY',outputCol='FLAG_OWN_REALTY_index',handleInvalid='keep')
NAME_INCOME_TYPE_indexer = StringIndexer(inputCol='NAME_INCOME_TYPE',outputCol='NAME_INCOME_TYPE_index',handleInvalid='keep')
NAME_EDUCATION_TYPE_indexer = StringIndexer(inputCol='NAME_EDUCATION_TYPE',outputCol='NAME_EDUCATION_TYPE_index',handleInvalid='keep')
NAME_FAMILY_STATUS_indexer = StringIndexer(inputCol='NAME_FAMILY_STATUS',outputCol='NAME_FAMILY_STATUS_index',handleInvalid='keep')
NAME_HOUSING_TYPE_indexer = StringIndexer(inputCol='NAME_HOUSING_TYPE',outputCol='NAME_HOUSING_TYPE_index',handleInvalid='keep')
OCCUPATION_TYPE_indexer = StringIndexer(inputCol='OCCUPATION_TYPE',outputCol='OCCUPATION_TYPE_index',handleInvalid='keep')
FLAG_MOBIL_indexer = StringIndexer(inputCol='FLAG_MOBIL',outputCol='FLAG_MOBIL_index',handleInvalid='keep')
FLAG_WORK_PHONE_indexer = StringIndexer(inputCol='FLAG_WORK_PHONE',outputCol='FLAG_WORK_PHONE_index',handleInvalid='keep')
FLAG_PHONE_indexer = StringIndexer(inputCol='FLAG_PHONE',outputCol='FLAG_PHONE_index',handleInvalid='keep')
FLAG_EMAIL_indexer = StringIndexer(inputCol='FLAG_EMAIL',outputCol='FLAG_EMAIL_index',handleInvalid='keep')
CARD_STATUS_indexer = StringIndexer(inputCol='CARD_STATUS',outputCol='CARD_STATUS_index',handleInvalid='keep')


In [0]:
#Performing OneHotEncoding
data_encoder = OneHotEncoder(inputCols=["CODE_GENDER_index",'FLAG_OWN_CAR_index','FLAG_OWN_REALTY_index',
                                       'NAME_INCOME_TYPE_index','NAME_EDUCATION_TYPE_index','NAME_FAMILY_STATUS_index',
                              'NAME_HOUSING_TYPE_index','OCCUPATION_TYPE_index','FLAG_MOBIL_index',"FLAG_WORK_PHONE_index","FLAG_PHONE_index","FLAG_EMAIL_index"], outputCols= ["CODE_GENDER_index_vec",'FLAG_OWN_CAR_index_vec','FLAG_OWN_REALTY_index_vec',
                                       'NAME_INCOME_TYPE_index_vec','NAME_EDUCATION_TYPE_index_vec','NAME_FAMILY_STATUS_index_vec',
                              'NAME_HOUSING_TYPE_index_vec','OCCUPATION_TYPE_index_vec','FLAG_MOBIL_index_vec',"FLAG_WORK_PHONE_index_vec","FLAG_PHONE_index_vec","FLAG_EMAIL_index_vec" ],
                                      handleInvalid='keep')

In [0]:
# Vector assembler is used to create a vector of input features
 
assembler = VectorAssembler(inputCols=["CODE_GENDER_index_vec",'FLAG_OWN_CAR_index_vec','FLAG_OWN_REALTY_index_vec',
                                       'NAME_INCOME_TYPE_index_vec','NAME_EDUCATION_TYPE_index_vec','NAME_FAMILY_STATUS_index_vec',
                              'NAME_HOUSING_TYPE_index_vec','OCCUPATION_TYPE_index_vec','FLAG_MOBIL_index_vec',"FLAG_WORK_PHONE_index_vec","FLAG_PHONE_index_vec","FLAG_EMAIL_index_vec","Age" ],
                            outputCol="features")

In [0]:
#Defining Logistic Regression Model
lr_model = LogisticRegression(labelCol='CARD_STATUS_index')

In [0]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data.

#building the PipeLine
pipe = Pipeline(stages=[CODE_GENDER_indexer,FLAG_OWN_CAR_indexer,FLAG_OWN_REALTY_indexer,NAME_INCOME_TYPE_indexer,NAME_EDUCATION_TYPE_indexer,NAME_FAMILY_STATUS_indexer,NAME_HOUSING_TYPE_indexer,OCCUPATION_TYPE_indexer,FLAG_MOBIL_indexer,FLAG_WORK_PHONE_indexer,FLAG_PHONE_indexer,FLAG_EMAIL_indexer,CARD_STATUS_indexer,data_encoder,assembler,lr_model])


In [0]:
#fitting pipe to train data
fit_model=pipe.fit(train_data)

In [0]:
#running modal on test data
results = fit_model.transform(test_data)

In [0]:
#predictions
results.select(['CARD_STATUS_index','prediction']).show()

+-----------------+----------+
|CARD_STATUS_index|prediction|
+-----------------+----------+
|              0.0|       0.0|
|              0.0|       0.0|
|              0.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              0.0|       0.0|
|              0.0|       0.0|
|              0.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              0.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              1.0|       0.0|
|              0.0|       0.0|
|              0.0|       0.0|
+-----------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
AUC_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='CARD_STATUS_index',metricName='areaUnderROC')

In [0]:
#Checking Area under the curve
AUC = AUC_evaluator.evaluate(results)

In [0]:
print("The area under the curve is {}".format(AUC))

The area under the curve is 0.500106863142958


In [0]:
#Modal Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
ACC_evaluator = MulticlassClassificationEvaluator(
    labelCol="CARD_STATUS_index", predictionCol="prediction", metricName="accuracy")

In [0]:
#printing the accuracy
accuracy = ACC_evaluator.evaluate(results)
print("The accuracy of the model is {}".format(accuracy))

The accuracy of the model is 0.9313794915138086


In [0]:
# Import the required libraries

from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import VectorAssembler,StringIndexer,StandardScaler
from pyspark.ml import Pipeline

In [0]:
#splitting data into train and test
train_data,test_data=data.randomSplit([0.7,0.3])

In [0]:
#String Indexer to represent the string features in the model.
CODE_GENDER_indexer = StringIndexer(inputCol='CODE_GENDER',outputCol='CODE_GENDER_index',handleInvalid='keep')
FLAG_OWN_CAR_indexer = StringIndexer(inputCol='FLAG_OWN_CAR',outputCol='FLAG_OWN_CAR_index',handleInvalid='keep')
FLAG_OWN_REALTY_indexer = StringIndexer(inputCol='FLAG_OWN_REALTY',outputCol='FLAG_OWN_REALTY_index',handleInvalid='keep')
NAME_INCOME_TYPE_indexer = StringIndexer(inputCol='NAME_INCOME_TYPE',outputCol='NAME_INCOME_TYPE_index',handleInvalid='keep')
NAME_EDUCATION_TYPE_indexer = StringIndexer(inputCol='NAME_EDUCATION_TYPE',outputCol='NAME_EDUCATION_TYPE_index',handleInvalid='keep')
NAME_FAMILY_STATUS_indexer = StringIndexer(inputCol='NAME_FAMILY_STATUS',outputCol='NAME_FAMILY_STATUS_index',handleInvalid='keep')
NAME_HOUSING_TYPE_indexer = StringIndexer(inputCol='NAME_HOUSING_TYPE',outputCol='NAME_HOUSING_TYPE_index',handleInvalid='keep')
OCCUPATION_TYPE_indexer = StringIndexer(inputCol='OCCUPATION_TYPE',outputCol='OCCUPATION_TYPE_index',handleInvalid='keep')
FLAG_MOBIL_indexer = StringIndexer(inputCol='FLAG_MOBIL',outputCol='FLAG_MOBIL_index',handleInvalid='keep')
FLAG_WORK_PHONE_indexer = StringIndexer(inputCol='FLAG_WORK_PHONE',outputCol='FLAG_WORK_PHONE_index',handleInvalid='keep')
FLAG_PHONE_indexer = StringIndexer(inputCol='FLAG_PHONE',outputCol='FLAG_PHONE_index',handleInvalid='keep')
FLAG_EMAIL_indexer = StringIndexer(inputCol='FLAG_EMAIL',outputCol='FLAG_EMAIL_index',handleInvalid='keep')
CARD_STATUS_indexer = StringIndexer(inputCol='CARD_STATUS',outputCol='CARD_STATUS_index',handleInvalid='keep')

In [0]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=["CODE_GENDER_index",'FLAG_OWN_CAR_index','FLAG_OWN_REALTY_index',
                                       'NAME_INCOME_TYPE_index','NAME_EDUCATION_TYPE_index','NAME_FAMILY_STATUS_index',
                              'NAME_HOUSING_TYPE_index','OCCUPATION_TYPE_index','FLAG_MOBIL_index',"FLAG_WORK_PHONE_index","FLAG_PHONE_index","FLAG_EMAIL_index"],
                            outputCol="unscaled_features")

In [0]:
#Creating StandardScalar object
scaler = StandardScaler(inputCol="unscaled_features",outputCol="features")

In [0]:
# Create an object for the Linear SVC model

svc_model = LinearSVC(labelCol='CARD_STATUS')

In [0]:
#defining pipeline
pipe = Pipeline(stages=[CODE_GENDER_indexer,FLAG_OWN_CAR_indexer,FLAG_OWN_REALTY_indexer,NAME_INCOME_TYPE_indexer,NAME_EDUCATION_TYPE_indexer,NAME_FAMILY_STATUS_indexer,NAME_HOUSING_TYPE_indexer,OCCUPATION_TYPE_indexer,FLAG_MOBIL_indexer,FLAG_WORK_PHONE_indexer,FLAG_PHONE_indexer,FLAG_EMAIL_indexer,CARD_STATUS_indexer,assembler,scaler,svc_model])


In [0]:
#fitting pipe to train data
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
fit_model=pipe.fit(train_data)

In [0]:
#running modal on test data
results = fit_model.transform(test_data)

In [0]:
results.select(['CARD_STATUS','prediction']).show()

+-----------+----------+
|CARD_STATUS|prediction|
+-----------+----------+
|          1|       1.0|
|          0|       1.0|
|          0|       1.0|
|          0|       1.0|
|          0|       1.0|
|          0|       1.0|
|          1|       1.0|
|          1|       1.0|
|          1|       1.0|
|          1|       1.0|
|          1|       1.0|
|          1|       1.0|
|          1|       1.0|
|          0|       1.0|
|          1|       1.0|
|          0|       1.0|
|          0|       1.0|
|          0|       1.0|
|          0|       1.0|
|          1|       1.0|
+-----------+----------+
only showing top 20 rows



In [0]:
#Evaluating the modal
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
ACC_evaluator = MulticlassClassificationEvaluator(
    labelCol="CARD_STATUS", predictionCol="prediction", metricName="accuracy")

In [0]:
#Printing the accuracy
accuracy = ACC_evaluator.evaluate(results)
print("The accuracy of the model is {}".format(accuracy))

The accuracy of the model is 0.9301366626270017
